# 🧠 Clasificación y Limpieza de Datos para Modelado Predictivo

## 📂 Carga del archivo CSV

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter.filedialog import askopenfilename

tk.Tk().withdraw()
file_path = askopenfilename(title="Selecciona el archivo CSV")
df_raw = pd.read_csv(file_path)
df_raw.head()

## 🎯 Crear variable binaria `Alta_conectividad`

In [ ]:
df_raw["Alta_conectividad"] = (df_raw["Horas_Internet"] > 3.5).astype(int)

## 🧩 Crear variable categórica `Nivel_conectividad`

In [ ]:
def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw["Nivel_conectividad"] = df_raw["Horas_Internet"].apply(clasificar_conectividad)

## 🧼 Imputación de valores faltantes

In [ ]:
from sklearn.impute import SimpleImputer
df_clean = df_raw.copy()

# Imputar Edad con mediana
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])

# Imputar Ingreso con media
imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

## 🔣 Codificación de variables categóricas

In [ ]:
df_clean = pd.get_dummies(df_clean, drop_first=True)

## 📏 Escalado solo de columnas numéricas válidas

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionar solo columnas numéricas válidas
cols_numericas = df_clean.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler()
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

## 🤖 Entrenamiento de modelo después de limpieza

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df_clean.drop(columns=["Alta_conectividad"])
y = df_clean["Alta_conectividad"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("✅ Exactitud del modelo:", round(accuracy, 4))